<center><h1><b>FineFoods Review Word to Vector</b></h1></center>
<center><h2><b>Recurrent Neural Network</b></h2></center>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [38]:
import pandas as pd
import numpy as np
import gensim as gsm

In [8]:
# The path to the text file
file_path = '/kaggle/input/graphs-online-reviews/web-FineFoods/foods.txt'

# Initialize lists to store data
data = []

# Temporary dictionary to store each review
review = {}


# Read the file line by line
with open(file_path, 'r', encoding='ISO-8859-1') as file:
    for line in file:
        line = line.strip()
        if not line:  # if it's a blank line, it means one review has ended
            if review:  # if review has data, append it to the list
                data.append(review)
                review = {}  # reset for the next review
        else:
            if ':' in line:  # Ensure the line contains a colon
                key, value = line.split(":", 1)  # split each line by the first colon
                review[key.strip()] = value.strip()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

In [17]:
df[:2]

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1.0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [10]:
df.shape

(568454, 8)

In [31]:
df['review/text'][0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

## Data Preprocessing

In [39]:
review_txt = df['review/text'].apply(gsm.utils.simple_preprocess)
review_txt

0         [have, bought, several, of, the, vitality, can...
1         [product, arrived, labeled, as, jumbo, salted,...
2         [this, is, confection, that, has, been, around...
3         [if, you, are, looking, for, the, secret, ingr...
4         [great, taffy, at, great, price, there, was, w...
                                ...                        
568449    [great, for, sesame, chicken, this, is, good, ...
568450    [disappointed, with, the, flavor, the, chocola...
568451    [these, stars, are, small, so, you, can, give,...
568452    [these, are, the, best, treats, for, training,...
568453    [am, very, satisfied, product, is, as, adverti...
Name: review/text, Length: 568454, dtype: object

In [40]:
# model parameters
model = gsm.models.Word2Vec(
window=5,
    min_count=2,
    workers=4
)

In [41]:
# Building vocabulary: these are unique list of words 
model.build_vocab(review_txt,progress_per=1000)

In [42]:
model.epochs

5

In [43]:
model.corpus_count

568454

In [44]:
# model training
model.train(review_txt, 
            total_examples=model.corpus_count, 
            epochs=model.epochs
           )

(163759961, 215303610)

In [45]:
# saving the model
model.save('Finefoods_RNN_word2vec_model.model')

In [50]:
# Eperimenting the model performance
model.wv.most_similar('bad')

[('terrible', 0.7092869877815247),
 ('horrible', 0.6844694018363953),
 ('good', 0.6778838634490967),
 ('gross', 0.6684685349464417),
 ('nasty', 0.6625246405601501),
 ('funny', 0.6516246795654297),
 ('shabby', 0.6490644812583923),
 ('weird', 0.6413177251815796),
 ('offensive', 0.6345061659812927),
 ('awful', 0.6175276041030884)]

In [51]:
# Similarity score of words
model.wv.similarity(w1='cheap', w2='inexpensive')

0.5085876

In [52]:
# Similarity score of words
model.wv.similarity(w1='great', w2='good')

0.7815019

In [57]:
# Similarity score of words
model.wv.similarity(w1='great', w2='product')

-0.1348509